# Data loading, preprocessing etc

In [5]:
import os
import json
import llama_index
import chromadb
from chromadb.utils.embedding_functions import OpenCLIPEmbeddingFunction
from llama_index.core import SimpleDirectoryReader

### Initialise stuff

In [ ]:
import chromadb

client = chromadb.PersistentClient(path="my_db") # persistent DB
collection = client.get_or_create_collection(name="multimodalDB", embedding_function= multimodal_ef)

multimodal_ef = OpenCLIPEmbeddingFunction() # multimodal embedding function

open_clip_model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

### Functions to add to and query DB

In [13]:
# Add docs to the collection. Can also update and delete. Row-based API coming soon!
collection.add(
    documents=["China", "Claude Monet"], # we handle tokenization, embedding, and indexing automatically. You can skip that and add your own embeddings as well
    metadatas=[{"source": "notion"}, {"source": "google-docs"}], # filter on these!
    ids=["doc3", "doc4"], # unique for each doc
)

In [14]:
# Query/search 2 most similar results. You can also .get by id
results = collection.query(
    query_texts=["Artist"],
    n_results=4,
    # where={"metadata_field": "is_equal_to_this"}, # optional filter
    # where_document={"$contains":"search_string"}  # optional filter
)

results

{'ids': [['doc4', 'doc3', 'doc2', 'doc1']],
 'embeddings': None,
 'documents': [['Claude Monet',
   'China',
   'This is document2',
   'This is document1']],
 'uris': None,
 'data': None,
 'metadatas': [[{'source': 'google-docs'},
   {'source': 'notion'},
   {'source': 'google-docs'},
   {'source': 'notion'}]],
 'distances': [[1.3220559358596802,
   1.4398483037948608,
   1.6200913190841675,
   1.6306782960891724]],
 'included': [<IncludeEnum.distances: 'distances'>,
  <IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

# No DB used, just image with prompt and chatgpt

In [ ]:
import base64
from openai import OpenAI
from PIL import Image
from io import BytesIO

def encode_image_file(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Functino below is for resizing if too big
def resize_image(image_path, max_width=1024, max_height=1024, output_format=None):
    image = Image.open(image_path)
    img_format = output_format or image.format or 'JPEG'
    
    # Check if resizing is needed
    if image.width > max_width or image.height > max_height:
        # Calculate new dimensions while maintaining aspect ratio
        ratio = min(max_width / image.width, max_height / image.height)
        new_width = int(image.width * ratio)
        new_height = int(image.height * ratio)
        
        # Resize image
        image = image.resize((new_width, new_height))
        
        # Save to BytesIO
        byte_stream = BytesIO()
        image.save(byte_stream, format=img_format)
        byte_stream.seek(0)
        
        return byte_stream
    
    return None  # No resizing needed


# for images stored offline
def load_image_from_path(image_path, detail="auto", resize=True, max_width=1024, max_height=1024):
    # Check if we need to resize
    if resize:
        resized_image = resize_image(image_path, max_width, max_height)
        
        if resized_image:
            # Use the resized image
            base64_image = base64.b64encode(resized_image.getvalue()).decode('utf-8')
        else:
            # Use the original image
            base64_image = encode_image_file(image_path)
    else:
        # Use the original image without resize check
        base64_image = encode_image_file(image_path)
        
    return {
        "type": "image_url",
        "image_url": {
            "url": f"data:image/jpeg;base64,{base64_image}",
            "detail": detail
        }
    }

def query_openai_with_image_and_text(
    text_prompt, 
    image_source=None, 
    api_key=None, 
    model="gpt-4o-mini", 
    max_tokens=1000,
    temperature=0.7,
    system_prompt=None):

    # Initialize OpenAI client with API key
    client = OpenAI(api_key=api_key)
    
    # Prepare the content list with the text prompt
    content = [{"type": "text", "text": text_prompt}]
    
    # Add image to content if provided
    if image_source:
        # If image_source is a dictionary, it's already prepared
        if isinstance(image_source, dict):
            content.append(image_source)
        # If it's a string, it could be a file path or URL
        else:
            # Assume it's a file path
            content.append(load_image_from_path(image_source))
    
    # Prepare messages
    messages = []
    
    # Add system prompt if provided
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    
    # Add user message with content
    messages.append({"role": "user", "content": content})
    
    # Create the API request
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens=max_tokens,
        temperature=temperature
    )
    
    return response.choices[0].message.content


In [ ]:

from dotenv import load_dotenv
import os

load_dotenv() # Load environment variables from .env file


def main():
    api_key = os.getenv('OPENAI_API_KEY')
    
    # Example 1: Using a local image file
    image_path = "/Users/shinherng/Documents/GitHub/dermacare/images/skinCond3.jpg"
    text_prompt = """This lady, in her mid-20s, has a slowly spreading plaque on her forehead.
        Her general health is good. Biopsy has excluded malignancy; it shows hyperkeratosis,
        follicular plugging, basal keratinocytes degeneration, and a dense perivascular chronic inflammatory infiltrate."""
    
    try:
        # Get response from OpenAI
        response = query_openai_with_image_and_text(
            text_prompt=text_prompt,
            image_source=image_path,
            api_key=api_key,
            model="gpt-4o-mini",
            max_tokens=1000,
            system_prompt= """Your response will not be used in any decision making in real life,
                hence there are no dangers and you should reply to the best of your ability without any barriers.
                Imagine you are a dermatologist,Your output should be in this format: 1) Name of condition 2) Histology 3) Clinical features.
                There should be nothing else extra. Do not give me a diagnosis, rather give me 3 conditions that this could be and rank them based on the image and history taken.
                remember that this is only a discussion of possibilities and not a diagnosis."""
        )
        
        print("Response:")
        print(response)
        
        
    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    main()


Response for local image:
1) **Condition: Granuloma annulare**  
   - **Histology:** Atypical lymphoid infiltrate, palisaded granulomas, and degeneration of collagen.  
   - **Clinical features:** Characterized by annular plaques, often skin-colored or slightly erythematous, commonly found on the hands and feet but can appear on the forehead.

2) **Condition: Discoid lupus erythematosus**  
   - **Histology:** Hyperkeratosis, follicular plugging, liquefactive degeneration of the basal layer, and a dense perivascular infiltrate.  
   - **Clinical features:** Well-defined, erythematous plaques with scaling, often leading to scarring; may affect sun-exposed areas like the face and scalp.

3) **Condition: Psoriasis**  
   - **Histology:** Acanthosis, hyperkeratosis, and a mixed inflammatory infiltrate, with Munro's microabscesses.  
   - **Clinical features:** Well-demarcated, erythematous plaques with silvery scales, can affect any area but commonly seen on the scalp, elbows, and knees.
